In [1]:
player_names = ["Wonde","Tome", "Hrici", "Kajlo", "Sasu", "OG"]

player_answers = [
    ["PER","ALO","VER","VER","DEV","SER","Ano","Ano","Nie","Nie","Ano","Nie","Nie","PER","Redbull"],
    ["PER","ALO","SAI","VER","DEV","SER","Ano","Ano","Ano","Ano","Ano","Nie","Nie","PER","Redbull"],
    ["PER","ALO","VER","VER","SAR","NOR","Ano","Ano","Ano","Nie","Ano","Nie","Nie","PER","Redbull"],
    ['ALO','PER','VER','VER','DEV','SAR','Ano','Ano','Ano','Nie','Ano','Nie','Ano-Redbull','ALO','Redbull'],
    ["PER","LEC","ALO","VER","SAR","TSU","Nie","Ano","Ano","Nie","Ano","Nie","Ano-Redbull","VER","Redbull"],
    ['PER','ALO','VER','PER','SAR','STR','Nie','Ano','Ano','Nie','Ano','Nie','Ano-Redbull','PER',"Redbull"]
]

ground_truth = ["PER", "VER", "ALO", "VER", "BOT", ["STR", "ALB"], "Nie", "Ano", "Nie", "Nie", "Ano", "Nie", "Ano-Redbull", "VER", "Redbull"]


def calculate_scores(player_names, player_answers, ground_truth):
    scores = {name: 0 for name in player_names}  # initialize scores to 0 for each player
    for i in range(len(ground_truth)):
        for name, answer in zip(player_names, player_answers):
            if i >= len(ground_truth) - 4:
                if answer[i] == ground_truth[i]:
                    scores[name] += 2  # add 2 points for a correct answer
            else:
                if type(answer[i]) == str and answer[i] == ground_truth[i]:   #signle correct answer
                        scores[name] += 2  # add 2 points for a correct answer
                elif type(answer[i]) == list and answer[i] in ground_truth[i]:   #multiple correct answers
                    scores[name] += 2  # add 2 points for a correct answer
                else:
                    scores[name] -= 1  # subtract 1 point for an incorrect answer
    return scores





scores = calculate_scores(player_names, player_answers, ground_truth)
print(scores)


{'Wonde': 11, 'Tome': 5, 'Hrici': 8, 'Kajlo': 7, 'Sasu': 18, 'OG': 10}


In [2]:
import pandas as pd

In [3]:
#nacitaj dotaznik

In [4]:
df = pd.read_csv(r'test.csv')

In [5]:
#pridaj odpovede z dotaznika do db

In [6]:
#treba vymazat riadky ktore nechcem ukladat

In [7]:
import sqlite3

grand_prix_id = 3

conn = sqlite3.connect('F1-2023.db')
c = conn.cursor()

for i in range(len(df['Meno'])):
    print()
    for n,ans in enumerate(df.loc[i][2:]):
        if type(ans) == str:
            #print(f'''{n+1}, {player_names.index(df.loc[i][1])+1}, {grand_prix_id}, {ans}''')
             c.execute(f'''
                      INSERT INTO bets (question_id, player_id, grand_prix_id, answer) 
                      VALUES ({n+1}, {player_names.index(df.loc[i][1])+1}, {grand_prix_id}, '{ans}')
                      ''')

conn.commit()

In [8]:
#print answers

In [9]:
c.execute('SELECT * FROM ground_truth')

myresult = c.fetchall()

for x in myresult:
  print(x)

In [10]:
c.execute('SELECT * FROM bets')

myresult = c.fetchall()

for x in myresult:
  print(x)

(1, '6', 3, 'Leclerc')
(2, '6', 3, 'Zhou')
(3, '6', 3, 'Alonso')
(4, '6', 3, 'Ocon')
(5, '6', 3, 'Leclerc')
(6, '6', 3, 'Sainz')
(7, '6', 3, 'Áno')
(8, '6', 3, 'Nie')
(9, '6', 3, 'Nie')
(11, '6', 3, 'Áno')
(12, '6', 3, 'Áno')
(14, '6', 3, 'Mercedes')
(15, '6', 3, 'Russell')
(16, '6', 3, 'Williams')
(1, '5', 3, 'Sainz')
(2, '5', 3, 'Russell')
(3, '5', 3, 'Alonso')
(4, '5', 3, 'Sainz')
(5, '5', 3, 'Hulkenberg')
(6, '5', 3, 'Bottas')
(7, '5', 3, 'Áno')
(8, '5', 3, 'Nie')
(9, '5', 3, 'Áno')
(10, '5', 3, 'Nie')
(11, '5', 3, 'Áno')
(12, '5', 3, 'Nie')
(13, '5', 3, 'Nie')
(14, '5', 3, 'Aston Martin')
(15, '5', 3, 'Stroll')


In [11]:
#insert ground_truth into database

In [12]:
ground_truth = ["PER", "VER", "ALO", "VER", "BOT", ["STR", "ALB"], "Nie", "Ano", "Nie", "Nie", "Ano", "Nie", "Ano-Redbull", "VER", "Redbull"]
grad_prix_id = 2
for n,ans in enumerate(ground_truth):
    if type(ans) == str:
        c.execute(f'''
              INSERT INTO ground_truth (grand_prix_id, question_id, correct_ans) 
              VALUES ({grand_prix_id}, {n+1}, '{ans}')
              ''')
    elif type(ans) == list:
        for ans1 in ans:
            c.execute(f'''
              INSERT INTO ground_truth (grand_prix_id, question_id, correct_ans) 
              VALUES ({grand_prix_id}, {n+1}, '{ans1}')
              ''')

In [13]:
#check number of correct answers

In [21]:
gp_id = 3
for i in range(len(player_names)):
    #correct answers
    c.execute(f'''SELECT COUNT(a.player_id) FROM bets a JOIN ground_truth b 
                 ON a.grand_prix_id = b.grand_prix_id 
                 AND a.answer = b.correct_ans
                 AND a.question_id = b.question_id
                 WHERE a.player_id = {i+1}
                 AND a.grand_prix_id = {gp_id}''')
    myresult = c.fetchall()
    correct_pts = myresult[0][0]*2
    #print(correct_pts)
    
    #incorrect answers
    c.execute(f'''SELECT a.question_id FROM bets a JOIN ground_truth b 
             ON a.grand_prix_id = b.grand_prix_id 
             AND a.answer <> b.correct_ans
             AND a.question_id = b.question_id
             WHERE a.player_id = {i+1}
             AND a.grand_prix_id = {gp_id}''')

    minus_pts = 0
    myresult1 = c.fetchall()
    for ans in myresult1:
        if ans[0] >= 1 and ans[0] <= 11:
            minus_pts += 1
    
    print(player_names[i])
    print(correct_pts - minus_pts)
            

Wonde
0
Tome
0
Hrici
0
Kajlo
0
Sasu
-7
OG
-8
